# 1.IMPORT LIBRARY

In [1]:
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
import os
import pandas as pd

import sys
# Tambahkan folder 'b/modules' ke sys.path

from modules.components import init_components


# 2.LOAD DATA


Langkah ini digunakan untuk memuat dataset **Customer Churn** ke dalam environment Python.

In [ ]:
data = pd.read_csv(r'D:\PROJECT-PORTOFOLIO\MLOPS\a443-cc-pipeline\data\Customer-Churn-Records.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain,Satisfaction Score,Card Type,Point Earned
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1,1,2,DIAMOND,464
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,1,3,DIAMOND,456
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1,1,3,DIAMOND,377
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0,0,5,GOLD,350
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,0,5,GOLD,425


## 2.1 Clean up irrelevant column

- RowNumber, CustomerId, dan Surname tidak mengandung informasi yang membantu model memprediksi churn.

- drop(columns=...) digunakan untuk menghapus kolom tersebut dari DataFrame.

- df.head() menampilkan 5 baris pertama dari dataset yang sudah dibersihkan.

In [ ]:
df = data.drop(columns=['RowNumber', 'CustomerId', 'Surname'])
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain,Satisfaction Score,Card Type,Point Earned
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1,1,2,DIAMOND,464
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,1,3,DIAMOND,456
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1,1,3,DIAMOND,377
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0,0,5,GOLD,350
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,0,5,GOLD,425


## 2.2 Chek the data

- melakukan chek terhadap data apakah terdapat missing value atau duplicate value

In [ ]:
print(df.duplicated().sum())
df.info()

0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   CreditScore         10000 non-null  int64  
 1   Geography           10000 non-null  object 
 2   Gender              10000 non-null  object 
 3   Age                 10000 non-null  int64  
 4   Tenure              10000 non-null  int64  
 5   Balance             10000 non-null  float64
 6   NumOfProducts       10000 non-null  int64  
 7   HasCrCard           10000 non-null  int64  
 8   IsActiveMember      10000 non-null  int64  
 9   EstimatedSalary     10000 non-null  float64
 10  Exited              10000 non-null  int64  
 11  Complain            10000 non-null  int64  
 12  Satisfaction Score  10000 non-null  int64  
 13  Card Type           10000 non-null  object 
 14  Point Earned        10000 non-null  int64  
dtypes: float64(2), int64(10), object(3)
memory usage: 1.

Okay, the data is clean from missing value and duplicate value. So, i will save to csv.

## 2.3 Ambil Sample

- Agar ringan saya hanya menggunakan sampel dari data

In [ ]:
new_df = pd.read_csv(r'D:\b\data\customer-churn.csv')
new_df = new_df.sample(n=1500, random_state=42)
data = new_df.to_csv('customer-churn.csv', index=False)

In [ ]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1500 entries, 1801 to 1070
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   CreditScore         1500 non-null   int64  
 1   Geography           1500 non-null   object 
 2   Gender              1500 non-null   object 
 3   Age                 1500 non-null   int64  
 4   Tenure              1500 non-null   int64  
 5   Balance             1500 non-null   float64
 6   NumOfProducts       1500 non-null   int64  
 7   HasCrCard           1500 non-null   int64  
 8   IsActiveMember      1500 non-null   int64  
 9   EstimatedSalary     1500 non-null   float64
 10  Exited              1500 non-null   int64  
 11  Complain            1500 non-null   int64  
 12  Satisfaction Score  1500 non-null   int64  
 13  Card Type           1500 non-null   object 
 14  Point Earned        1500 non-null   int64  
dtypes: float64(2), int64(10), object(3)
memory usage: 18

# 3. TAHAPAN PIPELINE ORCHESTRA

- PIPELINE_ROOT → tempat menyimpan artifact pipeline.

- InteractiveContext → jalankan pipeline TFX di notebook.

- init_components → buat semua komponen pipeline (preprocessing, trainer, evaluator, pusher).

In [3]:
# Direktori untuk menyimpan pipeline artifacts
PIPELINE_ROOT = "t"
os.makedirs(PIPELINE_ROOT, exist_ok=True)

# Buat context
context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

# Inisialisasi komponen
import os

BASE_DIR = r"d:\b\modules"

components = init_components(
    data_dir="data",
    transform_module=os.path.join(BASE_DIR, "customer_churn_transform.py"),
    training_module=os.path.join(BASE_DIR, "customer_churn_trainer.py"),
    training_steps=100,
    eval_steps=50,
    serving_model_dir="model_serving"
)


## 3.1 ExampleGEN

- ExampleGen mengambil dataset dari folder data dan membuat TFRecord examples untuk pipeline.

- context.run(example_gen) menjalankan komponen ini di notebook.

- Outputnya akan digunakan oleh komponen selanjutnya seperti Transform dan Trainer.

In [4]:
# 1. ExampleGen
example_gen = components[0]
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

## 3.2 StatisticGen

- StatisticsGen menghitung statistik deskriptif dari dataset (misal mean, min, max, missing values).

- context.run(statistics_gen) menjalankan komponen ini di notebook.

- Outputnya membantu memahami distribusi data sebelum preprocessing atau training.

In [5]:
statistics_gen = components[1]
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

## 3.3 SchemaGen

- SchemaGen membuat schema otomatis berdasarkan statistik data (tipe data, domain nilai, dll.).

- context.run(schema_gen) menjalankan komponen ini di notebook.

- Schema dipakai untuk validasi data dan preprocessing.

In [6]:
schema_gen = components[2]
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

## 3.4 ExampleValidator

- ExampleValidator memeriksa data terhadap schema (misal missing value, nilai anomali, atau outlier).

- context.run(example_validator) menjalankan validasi tersebut.

- Tujuannya memastikan data bersih dan konsisten sebelum masuk ke tahap transformasi.

In [7]:
example_validator = components[3]
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

## 3.5 Transform

- Transform melakukan feature engineering & preprocessing sesuai modul customer_churn_transform.py.

- context.run(transform) mengeksekusi proses transformasi data.

- Hasilnya berupa dataset yang sudah siap digunakan untuk training model.

In [8]:
transform = components[4]
context.run(transform)

Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.


INFO:tensorflow:Assets written to: t\Transform\transform_graph\5\.temp_path\tftransform_tmp\7710e8c13a6a4dadaed999b8c8fa9a08\assets


INFO:tensorflow:Assets written to: t\Transform\transform_graph\5\.temp_path\tftransform_tmp\7710e8c13a6a4dadaed999b8c8fa9a08\assets


INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

## 3.6 Trainer

- Trainer melatih model menggunakan data yang sudah ditransformasi.

- Script training didefinisikan di customer_churn_trainer.py.

- context.run(trainer) mengeksekusi proses training dan menghasilkan model terlatih.

In [9]:
trainer = components[5]
context.run(trainer)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Geography_xf (InputLayer)      [(None, 4)]          0           []                               
                                                                                                  
 Gender_xf (InputLayer)         [(None, 3)]          0           []                               
                                                                                                  
 Card Type_xf (InputLayer)      [(None, 5)]          0           []                               
                                                                                                  
 CreditScore_xf (InputLayer)    [(None, 1)]          0           []                               
                                                                                              

INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:Assets written to: t\Trainer\model\6\Format-Serving\assets


INFO:tensorflow:Assets written to: t\Trainer\model\6\Format-Serving\assets


ExecutionResult(
    component_id: Trainer
    execution_id: 6
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

## 3.7 ModelResolver

- ModelResolver memilih model terbaru (baseline) untuk dibandingkan dengan model yang baru dilatih.

- context.run(model_resolver) menjalankan komponen ini.

- Tujuannya memastikan hanya model terbaik yang akan dilanjutkan ke tahap evaluasi & deployment.

In [10]:
model_resolver = components[6]
context.run(model_resolver)

ExecutionResult(
    component_id: latest_blessed_model_resolver
    execution_id: 7
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={})
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}))

## 3.8 Evaluator

- Evaluator membandingkan model baru dengan model baseline menggunakan metrik evaluasi.

- context.run(evaluator) menjalankan proses evaluasi tersebut.

- Tujuannya memastikan model baru memiliki performa lebih baik sebelum didorong ke tahap deployment

In [13]:
evaluator = components[7]
context.run(evaluator)

ExecutionResult(
    component_id: Evaluator
    execution_id: 10
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={})
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}))

## 3.9 Pusher

- Pusher menyimpan dan men-deploy model yang lolos evaluasi ke folder model_serving.

- context.run(pusher) mengeksekusi proses deployment model.

- Tujuannya agar model siap digunakan untuk serving/inference.

In [14]:
pusher = components[8]
context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 11
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}))